<h1>US Airline fuel usage</h1>

This notebook obtains the fuel usage by US airline carriers by month and compares the figures from last year and an average across previous years to view the impact of Covid.

This first requires the scraping of data from the table reported at <a href="https://www.transtats.bts.gov/fuel.asp">https://www.transtats.bts.gov/fuel.asp</a>

<h2>Obtaining data</h2>

In [1]:
# Import required packages
import requests
import lxml.html as lh
import pandas as pd

In [2]:
# Obtain page html
url = "https://www.transtats.bts.gov/fuel.asp"
html = requests.get(url, verify=False)

/Users/pietroservini/opt/anaconda3/envs/data-vis/lib/python3.6/site-packages/urllib3/connectionpool.py:1020: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.transtats.bts.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


In [3]:
doc = lh.fromstring(html.content)

In [4]:
# Get info between <tr>...</tr>
row_elements = doc.xpath('//tr')

In [5]:
# We are only interested in those rows that have 11 columns
rel_rows = [t for t in row_elements if len(t) == 11]

In [6]:
# Define the names of the columns in our DataFrame
col_names = ['year', 'month', 
             'consumption_domestic', 'cost_domestic', 'cost_per_gallon_domestic',
             'consumption_international', 'cost_international', 'cost_per_gallon_international',
             'consumption_total', 'cost_total', 'cost_per_gallon_total']

In [7]:
# DataFrame will be created from a list of lists
l_of_l = []
for row in rel_rows:
    l = []
    for d in row:
        data = d.text_content()
        l.append(data)
    l_of_l.append(l)

In [8]:
df = pd.DataFrame(l_of_l, columns=col_names)

In [9]:
df.head()

,year,month,consumption_domestic,cost_domestic,cost_per_gallon_domestic,consumption_international,cost_international,cost_per_gallon_international,consumption_total,cost_total,cost_per_gallon_total
0,2000,January,"1,108.6",766.5,0.69,386.8,313.6,0.81,"1,495.4","1,080.1",0.72
1,2000,February,"1,068.4",772.3,0.72,366.8,298.7,0.81,"1,435.2","1,071.1",0.75
2,2000,March,"1,174.3",875.5,0.75,399.7,330.1,0.83,"1,574.0","1,205.6",0.77
3,2000,April,"1,145.3",841.9,0.74,407.9,336.7,0.83,"1,553.1","1,178.7",0.76
4,2000,May,"1,174.2",833.3,0.71,434.4,337.9,0.78,"1,608.6","1,171.2",0.73


In [10]:
# Convert the relevant fields to numeric values
num_cols = ['year', 
            'consumption_domestic', 'cost_domestic', 'cost_per_gallon_domestic', 
            'consumption_international', 'cost_international', 'cost_per_gallon_international', 
            'consumption_total', 'cost_total', 'cost_per_gallon_total']
for col in num_cols:
    # remove any comma first
    df[col] = df[col].apply(lambda x: str(x).replace(',', ''))
    df[col] = pd.to_numeric(df[col])

In [11]:
df.columns

Index(['year', 'month', 'consumption_domestic', 'cost_domestic',
       'cost_per_gallon_domestic', 'consumption_international',
       'cost_international', 'cost_per_gallon_international',
       'consumption_total', 'cost_total', 'cost_per_gallon_total'],
      dtype='object')

In [12]:
# Retain only columns I want to plot
df = df[['year', 'month', 'consumption_domestic', 'consumption_international', 'cost_per_gallon_total']]

<h2>Plot</h2>

We'll create a plot that compares 2019 with 2020, showing domestic and international consumption as separate lines.  We also want to see what the cost of oil was too.

In [13]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [14]:
df_20 = df[df.year==2020]
df_19 = df[df.year==2019]

In [15]:
# Compute ratio of 2019 and 2020 figures
df_comp = df_19.merge(df_20, on='month').rename(columns={'consumption_domestic_x': 'domestic_2019',
                                                        'consumption_domestic_y': 'domestic_2020',
                                                        'consumption_international_x': 'international_2019',
                                                        'consumption_international_y': 'international_2020',
                                                        'cost_per_gallon_total_x': 'cost_per_gallon_2019',
                                                        'cost_per_gallon_total_y': 'cost_per_gallon_2020'})
df_comp.drop(['year_x', 'year_y'], axis=1, inplace=True)
df_comp['domestic_ratio'] = df_comp.domestic_2020/df_comp.domestic_2019 - 1
df_comp['international_ratio'] = df_comp.international_2020/df_comp.international_2019 - 1
df_comp['cost_rebased'] = df_comp['cost_per_gallon_2020']/df_comp.loc[df_comp[df_comp.month=='December'].index.values[0], 'cost_per_gallon_2019']
df_comp['cost_rebased'] = df_comp.cost_rebased - 1
df_comp['month'] = df.month.map(lambda x: x[:3].upper())

In [17]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=df_comp.month,
                        y=df_comp.domestic_ratio,
                        line=dict(color='rgb(25,25,112)'),
                        name='Domestic flights',
                        fill='tozeroy',
                        fillcolor='rgba(25,25,112,0.1)'))

fig.add_trace(go.Scatter(x=df_comp.month,
                        y=df_comp.international_ratio,
                        line=dict(color='rgb(0,191,255)'),
                        name='International flights',
                        fill='tozeroy',
                        fillcolor='rgba(0,191,255,0.1)'))

fig.add_trace(go.Scatter(x=df_comp.month,
                         y=df_comp.cost_rebased,
                         line=dict(color='black', dash='dash'),
                         name='Cost per gallon, 2020 <br> (% based on December 2019 level)'))

fig.update_yaxes(title='% increase/decrease on 2019 levels', 
                 range=[-1.0, 0.1], 
                 tickformat=',.0%',
                 showgrid=True,
                 gridcolor='rgb(224,224,224)',
                 zeroline=True,
                 zerolinecolor='black',
                 zerolinewidth=1)

fig.update_xaxes(range=[-0.3,11.1])

fig.update_layout(
    title=dict(text='Clipped wings' + '<br>' + 
               '<span style="font-size: 16px;">The impact of Covid-19 on the amount of fuel consumed by aeroplanes belonging to US airline companies</span>',
               font=dict(size=24, color='black')),
    paper_bgcolor='white',
    plot_bgcolor='white',
    hovermode="x unified"
)

fig.add_annotation(text='Source: <a href="https://www.transtats.bts.gov/fuel.asp">https://www.transtats.bts.gov/fuel.asp</a>',
                   font=dict(size=12),
                  xref='paper', yref='paper',
                  x=1.25, y=-0.1,
                  showarrow=False
                  )

fig.write_html('us_airline_fuel_consumption.html')
fig.show()

In [ ]:
df_comp[['month', 'cost_per_gallon_total_x', 'cost_per_gallon_total_y']]